In [1]:
import time
import numpy as np
from src.data_loader import load_data
from src.feature_engineering import average_daily
from src.model.train_model import get_X_y, train_and_evaluate_models
import os
from objective_function import objective_et
from src.Optimiser.KOA.koa_optimizer import koa_optimizer  # اگر اسم تابع اصلی الگوریتم KOA هست
from src.Optimiser.COA.coa_optimizer import coa_optimizer  # اگر اسم تابع اصلی الگوریتم COA هست

DATA_PATH = os.path.join("data", "Data.xlsx")


In [ ]:


print("🔹 Loading raw data...")

df = load_data(DATA_PATH)
print(f"✅ Raw data shape: {df.shape}")

print("🔹 Aggregating daily averages (96 samples per day)...")
df_avg = average_daily(df, samples_per_day=96)

print(f"✅ Aggregated data shape: {df_avg.shape}")
print("✅ Sample preview:")
print(df_avg.head())
print("🔹 Preparing features and target...")
X, y = get_X_y(df_avg)

print(f"✅ X shape: {X.shape}, y shape: {y.shape}")

print("🔹 Training models with 5-Fold CV...")
start_time = time.time()  # Start timing
etr_scores, lasso_scores = train_and_evaluate_models(X, y)
elapsed_time = time.time() - start_time  # Calculate elapsed time
print(f"\n⏱️ Total training and evaluation time: {elapsed_time:.2f} seconds")
print("\n📊 Final Results (Averaged Metrics):")
def summarize_metrics(metrics_list):
    return {key: np.mean([m[key] for m in metrics_list]) for key in metrics_list[0]}

avg_etr = summarize_metrics(etr_scores)
avg_lasso = summarize_metrics(lasso_scores)

print("🔹 Extra Trees Regressor:")
for k, v in avg_etr.items():
    print(f"  {k}: {v:.4f}")

print("\n🔹 Lasso Regression:")
for k, v in avg_lasso.items():
    print(f"  {k}: {v:.4f}")
    # تعریف کران‌ها برای 4 پارامتر Extra Trees
    # [n_estimators, max_depth, min_samples_split, min_samples_leaf]
print("\n🔹 Starting KOA optimization for Extra Trees Regressor...")
lb = [10, 5, 2, 1]
ub = [300, 50, 10, 10]
dim = 4
n_agents = 10
max_iter = 20
best_params, best_mae = koa_optimizer(objective_et, lb, ub, dim, n_agents, max_iter, X, y)
print("\n🏆 Best parameters:", best_params)
print("📉 Best MAE:", best_mae)

# تعریف کران‌ها برای 4 پارامتر Extra Trees
# [n_estimators, max_depth, min_samples_split, min_samples_leaf]
# starting with COA optimizer
print("\n🔹 Starting COA optimization for Extra Trees Regressor...")
lb_coa = [10, 5, 2, 1]
ub_coa = [300, 50, 10, 10]
dim_coa = 4
n_agents_coa = 10
max_iter_coa = 20
best_params_coa, best_mae_coa = coa_optimizer(objective_et, lb_coa, ub_coa, dim_coa, n_agents_coa, max_iter_coa, X, y)

print("\n🏆 Best parameters (COA):", best_params_coa)
print("📉 Best MAE (COA):", best_mae_coa)
import matplotlib.pyplot as plt
from analysis.wilcoxon_test import perform_wilcoxon_test
# Perform Wilcoxon test between the two models
mae_etr = [score['MAE'] for score in etr_scores]
mae_lr = [score['MAE'] for score in lasso_scores]
perform_wilcoxon_test(mae_etr, mae_lr, model_name_1="ETR", model_name_2="LR")


